In [1]:
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import pandas as pd

df = pd.read_parquet('train.parquet')

ssubmission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')

In [2]:
targets = ssubmission.set_index('id').columns

In [3]:
for col in targets:
    df[col] = 0

In [5]:
from collections.abc import Iterable
import numpy as np
for ind, row in df.iterrows():
    if isinstance(row['techniques'], Iterable):
        for t in row['techniques']:
            df.loc[ind, t] = 1

In [6]:
y_true = df[targets].values

In [7]:
y_zero = np.zeros_like(y_true)
y_one = np.ones_like(y_true)

In [10]:
y_true.mean(axis=0)

array([0.03610675, 0.07849294, 0.1007326 , 0.04107797, 0.04133961,
       0.51622187, 0.12637363, 0.12087912, 0.13396128, 0.12114076])

In [12]:
# Define 5-Fold Multilabel Stratification
n_splits = 5
mlskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Splitting the dataset
test_indexes = []
for fold, (train_idx, test_idx) in enumerate(mlskf.split(df, y_true)):
    test_indexes.append(test_idx)

In [16]:
df['fold'] = -1

In [17]:
for ind, test_idx in enumerate(test_indexes):
    df.loc[test_idx, 'fold'] = ind

In [19]:
df.fold.value_counts()

fold
1    765
3    765
2    764
0    764
4    764
Name: count, dtype: int64

In [20]:
for fold_ind in range(5):
    mask = (df.fold == fold_ind).values
    print(fold_ind)
    print(y_true[mask].mean(axis=0))

0
[0.03664921 0.07853403 0.10078534 0.04057592 0.04188482 0.51570681
 0.12696335 0.12041885 0.13350785 0.12172775]
1
[0.03529412 0.07843137 0.10065359 0.04183007 0.04052288 0.51503268
 0.1254902  0.12026144 0.13464052 0.12026144]
2
[0.03534031 0.07853403 0.10078534 0.04057592 0.04188482 0.51701571
 0.12565445 0.12041885 0.13350785 0.12041885]
3
[0.03660131 0.07843137 0.10065359 0.04183007 0.04183007 0.51633987
 0.12679739 0.12156863 0.13333333 0.12156863]
4
[0.03664921 0.07853403 0.10078534 0.04057592 0.04057592 0.51701571
 0.12696335 0.12172775 0.13481675 0.12172775]


In [22]:
df['content_len'] = df.content.str.len()
df.groupby('fold').content_len.mean(), df.groupby('fold').content_len.median()

(fold
 0    620.276178
 1    618.558170
 2    573.675393
 3    637.626144
 4    588.374346
 Name: content_len, dtype: float64,
 fold
 0    432.0
 1    407.0
 2    385.0
 3    462.0
 4    403.0
 Name: content_len, dtype: float64)

In [29]:
df['trigger_words_len'] = df.trigger_words.apply(lambda x: sum(y[1] - y[0] for y in x) if x is not None else 0)
df.groupby('fold').trigger_words_len.mean(), df.groupby('fold').trigger_words_len.median()

(fold
 0    146.578534
 1    145.339869
 2    153.742147
 3    150.724183
 4    154.413613
 Name: trigger_words_len, dtype: float64,
 fold
 0    77.0
 1    75.0
 2    67.0
 3    72.0
 4    72.0
 Name: trigger_words_len, dtype: float64)

In [31]:
df[['id', 'fold']].to_csv('cv_split.csv', index=False)